In [3]:
# import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
import pingouin as pg
import researchpy

%matplotlib inline

In [1]:
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本

0.22.1


In [4]:
#讀入資料
df_train = pd.read_csv('HWData/Titanic_train.csv')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [6]:
#遺失值刪除
complete_data=df_train[['Age','Survived','Sex']].dropna()
display(complete_data)

,Age,Survived,Sex
0,22.0,0,male
1,38.0,1,female
2,26.0,1,female
3,35.0,1,female
4,35.0,0,male
...,...,...,...
885,39.0,0,female
886,27.0,0,male
887,19.0,1,female
889,26.0,1,male


In [7]:
#Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。
# 運用 apply function  做變數轉換
complete_data['Age_above65_']=complete_data['Age'].apply((lambda x: 'Y' if x >=65 else 'N'))
display(complete_data.head(5))

,Age,Survived,Sex,Age_above65_
0,22.0,0,male,N
1,38.0,1,female,N
2,26.0,1,female,N
3,35.0,1,female,N
4,35.0,0,male,N


In [28]:
#Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。

def above65_female_check(row):
    if row.Sex =='female' or row.Age_above65_ =='Y':
        return 'Y'
    else:
        return 'N'

complete_data['Age_above65_female'] = complete_data.apply(above65_female_check, axis =1)

In [29]:
complete_data

,Age,Survived,Sex,Age_above65_,Age_above65_female
0,22.0,0,male,N,N
1,38.0,1,female,N,Y
2,26.0,1,female,N,Y
3,35.0,1,female,N,Y
4,35.0,0,male,N,N
...,...,...,...,...,...
885,39.0,0,female,N,Y
886,27.0,0,male,N,N
887,19.0,1,female,N,Y
889,26.0,1,male,N,N


In [30]:
#Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?
#2新增變數皆為離散型，目標變變亦是離散型，使用Cramer’s V 係數以判斷

contTable = pd.crosstab(complete_data['Age_above65_'], complete_data['Survived'])
contTable

Survived,0,1
Age_above65_,,
N,414,289
Y,10,1


In [31]:
dof = min(contTable.shape[0], contTable.shape[1]) - 1
dof

1

In [32]:
crosstab, res = researchpy.crosstab(complete_data['Age_above65_'], complete_data['Survived'], test='chi-square')
print(res.loc[2,'results'])
#Negligible

0.0803


In [33]:
contTable2 = pd.crosstab(complete_data['Age_above65_female'], complete_data['Survived'])
contTable2

Survived,0,1
Age_above65_female,,
N,350,92
Y,74,198


In [34]:
dof2 = min(contTable2.shape[0], contTable2.shape[1]) - 1
dof2

1

In [36]:
crosstab2, res2 = researchpy.crosstab(complete_data['Age_above65_female'], complete_data['Survived'], test='chi-square')
print(res2.loc[2,'results'])
#Large

0.514
